# AIOE 데이터 분석 및 시각화 (EDA)

이 노트북에서는 02번에서 계산된 AIOE 점수를 다양한 관점에서 **탐색**하고 **시각화**합니다.

## EDA(Exploratory Data Analysis)란?

**탐색적 데이터 분석** - 데이터를 여러 각도에서 살펴보고 패턴을 발견하는 과정

**이 노트북의 목표:**
- 📊 AIOE 점수가 어떻게 분포되어 있나?
- 🏆 어떤 직업이 AI에 가장 많이/적게 노출되나?
- 🏢 어떤 직업군이 AI 영향을 많이 받나?

## 사용할 분석 기법

1. **기술 통계** - 평균, 표준편차, 최소/최대값
2. **시각화** - 히스토그램, 박스플롯, 바차트
3. **그룹 비교** - 직업군별 AIOE 비교

## 0. 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install koreanize_matplotlib -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 그래프 한글 설정
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 데이터 경로 설정
PROCESSED_DIR = '/content/drive/MyDrive/aioe_project/datas/processed'

---

## 1. 데이터 로드 및 확인

### 사용할 데이터

02번 노트북에서 계산한 **직업별 AIOE 점수 데이터**를 사용합니다:

**aioe_master.csv**
- **894개 행** (894개 직업)
- **8개 컬럼**:
  - `soc_code`: 직업 코드
  - `title`: 직업명
  - `total_weight_impact`: AI 영향도 총합
  - `ability_base`: 능력 가중치 총합
  - `n_abil`: 능력 개수 (52개)
  - `abil_base_impact`: AIOE 기본 점수 (0~1)
  - `aioe`: AIOE 표준화 점수 (Z-score)
  - `rank`: AIOE 순위 (1~894)

In [ ]:
# 데이터 로드
df = pd.read_csv(f'{PROCESSED_DIR}/aioe_master.csv')

In [ ]:
df.tail()

,soc_code,Title,total_weight_impact,ability_base,n_abil,abil_base_impact,aioe,rank
889,53-7071.00,Gas Compressor and Gas Pumping Station Operators,5.580818,10.913931,52,0.511348,-0.975348,685
890,53-7072.00,"Pump Operators, Except Wellhead Pumpers",5.979743,11.795703,52,0.506942,-1.027508,704
891,53-7073.00,Wellhead Pumpers,4.797749,9.752697,52,0.491941,-1.205123,763
892,53-7081.00,Refuse and Recyclable Material Collectors,4.024189,8.612983,52,0.467224,-1.497766,846
893,53-7121.00,"Tank Car, Truck, and Ship Loaders",5.912775,12.689986,52,0.465940,-1.512960,850


In [ ]:
# Title 소문자로 변경
df = df.rename(columns={'Title': 'title'})
df.head(1)

,soc_code,title,total_weight_impact,ability_base,n_abil,abil_base_impact,aioe,rank
0,11-1011.00,Chief Executives,7.771722,11.088054,52,0.700909,1.268992,106


In [ ]:
# 컬럼 순서 재배치 (rank, soc_code, title..)

df = df[[
    'rank', 'soc_code', 'title', 'aioe', 'abil_base_impact',
    'total_weight_impact', 'ability_base', 'n_abil'
]]

### 📊 기술 통계 해석

**과학적 표기법:**
- `-3.8e-16` = -3.8 × 10⁻¹⁶ = -0.00000000000000038
- 사실상 **0** (컴퓨터 계산의 부동소수점 오차)

**aioe 컬럼 주요 지표:**
- **mean = 0**: Z-score의 특성 (표준화된 점수)
- **std = 1.00**: Z-score의 특성 (표준편차 = 1)
- **50% (중앙값) = 0.02**: 절반은 양수, 절반은 음수
- **min = -1.97, max = 1.79**: 모든 직업이 ±2 범위 내 (정상 범위)

---

## 2. AIOE 점수 분포 분석

### 2.1 전체 분포 확인

894개 직업의 AIOE 점수가 어떻게 분포되어 있는지 살펴봅니다.

**Z-score 특성:**
- 평균 = 0
- 표준편차 = 1
- 대부분 -2 ~ +2 사이에 분포
- 양수 = 평균보다 높음 (AI 노출도 높음)
- 음수 = 평균보다 낮음 (AI 노출도 낮음)

### 📊 차트 해석

**히스토그램**
- **이봉분포(Bimodal Distribution)** 형태
- 두 개의 봉우리가 관찰됨:
  - 음수 구간 (-0.8 ~ -1.0 근처): AI 노출도 낮은 직업군
  - 양수 구간 (+0.8 ~ +1.2 근처): AI 노출도 높은 직업군
- 중간 구간(0 근처)의 직업이 상대적으로 적음

### 2.2 AI 노출도 TOP 10 & BOTTOM 10

**rank 컬럼의 의미:**
- rank = 1: AIOE가 가장 높은 직업 (AI 노출도 최고)
- rank = 894: AIOE가 가장 낮은 직업 (AI 노출도 최저)

---

## 3. 직업군별 AIOE 비교

### 3.1 SOC 대분류 (22개 직업군)

O*NET의 **SOC(Standard Occupational Classification)** 코드 체계를 사용하여 894개 직업을 22개 대분류로 그룹화합니다.

**SOC 코드 형식:**
- `11-1011.00` → **11**(대분류) - 1011(세부직업) . 00(전문화)
- 앞 2자리가 직업군을 나타냄

**분석 목표:**
- 어떤 직업군이 AI에 가장 많이 노출되나?
- 직업군 간 차이는 얼마나 크나?

In [ ]:
# SOC 대분류 이름 매핑
# https://www.bls.gov/soc/2018/major_groups.htm
soc_major_names = {
    '11': '경영', '13': '사무/금융', '15': '컴퓨터/수학', '17': '건축/공학',
    '19': '과학', '21': '지역사회/사회', '23': '법률', '25': '교육',
    '27': '예술/디자인/미디어', '29': '보건', '31': '보건지원', '33': '보안',
    '35': '음식서비스', '37': '건물관리', '39': '개인서비스', '41': '판매',
    '43': '사무지원', '45': '농림어업', '47': '건설', '49': '설치/정비',
    '51': '생산', '53': '운송'
}

# SOC 대분류 추출
df['soc_major'] = df['soc_code'].str[:2]
df['soc_major_name'] = df['soc_major'].map(soc_major_names)

In [ ]:
df.head()

,rank,soc_code,title,aioe,abil_base_impact,total_weight_impact,ability_base,n_abil,soc_major,soc_major_name
0,106,11-1011.00,Chief Executives,1.268992,0.700909,7.771722,11.088054,52,11,경영
1,69,11-1011.03,Chief Sustainability Officers,1.349286,0.707691,6.363911,8.992497,52,11,경영
2,199,11-1021.00,General and Operations Managers,1.029288,0.680664,5.729594,8.417660,52,11,경영
3,156,11-2011.00,Advertising and Promotions Managers,1.142671,0.690240,6.281507,9.100466,52,11,경영
4,161,11-2021.00,Marketing Managers,1.129040,0.689089,6.247070,9.065697,52,11,경영


In [ ]:
# 직업군별 통계


**표준편차(std)의 의미:**
- **작음 (법률 0.17)**: 직업군 내 직업들이 비슷한 AIOE
  - 예: 변호사, 판사 모두 AIOE 높음 (균일)
- **큼 (예술/디자인 0.83)**: 직업군 내 편차 큼
  - 예: 작가(AIOE 높음) vs 무용수(AIOE 낮음)

**샘플 수(count):**
- 법률 7개 vs 생산 107개
- 샘플이 많을수록 평균의 **신뢰성** 높음
- 법률 직군은 샘플이 적어 해석 시 주의 필요

### 3.2 직업군별 평균 AIOE 비교

---

## 4. 결론

### 핵심 발견

1. **AIOE 분포**
   - 이봉분포(Bimodal) 형태: 양극화 현상
   - AI 노출도가 높은 직업과 낮은 직업이 명확히 구분됨

2. **AI 노출도가 높은 직업**
   - 교육 관련 직업 (교수, 교사)
   - 사무/금융 관련 직업
   - 법률 관련 직업
   - **공통점**: 언어/정보 처리가 핵심인 직업

3. **AI 노출도가 낮은 직업**
   - 건설 관련 직업
   - 생산/제조 관련 직업
   - 농림어업 관련 직업
   - **공통점**: 신체적/물리적 작업이 핵심인 직업

4. **직업군별 차이**
   - 법률, 교육, 사무/금융 직군이 높은 AIOE (평균 > 1.0)
   - 건설, 생산, 운송 직군이 낮은 AIOE (평균 < -0.9)
   - 같은 직업군 내에서도 편차가 존재 (표준편차 참고)

### 인사이트

- **언어 모델 AI는 화이트칼라 직업에 더 큰 영향**을 미칩니다
- 단순히 "고학력 직업"이 아니라 "언어/사고 능력 중심 직업"이 영향을 많이 받습니다
- 신체적/물리적 작업은 현재 언어 모델 AI로는 대체하기 어렵습니다
- 직업 시장의 **양극화 구조**가 AIOE 분포에 명확히 반영됨